In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import random

from sklearn.preprocessing import LabelEncoder

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def seed_everything(seed=999):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(49)

In [ ]:
drug_featuers = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
drug_featuers.head()

In [ ]:
fet_g = list(drug_featuers)[4:776]
fet_c = list(drug_featuers)[776:876]
drug_featuers['gsum'] = drug_featuers[fet_g].sum(axis=1)
drug_featuers['gstd'] = drug_featuers[fet_g].std(axis=1)
drug_featuers['gmean'] = drug_featuers[fet_g].mean(axis=1)
drug_featuers['gkurtosis'] = drug_featuers[fet_g].kurtosis(axis=1)
drug_featuers['gskew'] = drug_featuers[fet_g].skew(axis=1)

drug_featuers['csum'] = drug_featuers[fet_c].sum(axis=1)
drug_featuers['cstd'] = drug_featuers[fet_c].std(axis=1)
drug_featuers['cmean'] = drug_featuers[fet_c].mean(axis=1)
drug_featuers['ckurtosis'] = drug_featuers[fet_c].kurtosis(axis=1)
drug_featuers['cskew'] = drug_featuers[fet_c].skew(axis=1)


In [ ]:
drug_featuers.shape

In [ ]:
drug_featuers.shape

In [ ]:
drug_y = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
drug_y.head()

In [ ]:
drug_y2 = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
drug_y2.head()

In [ ]:
#merge 2 dataframes based on one column
drugs_all = pd.merge(drug_featuers, drug_y, on='sig_id')

drugs_all.head()

In [ ]:
drugs_all2 = pd.merge(drug_featuers, drug_y2, on='sig_id')

drugs_all2.head()

In [ ]:
x = drugs_all.iloc[:, 1:drug_featuers.shape[1]].values

x

In [ ]:
y = drugs_all.iloc[:, drug_featuers.shape[1]:].values

y

In [ ]:
y2 = drugs_all2.iloc[:, drug_featuers.shape[1]:].values

y2

In [ ]:
x[:,1] = x[:,1] / 72
labelencoder_x2 = LabelEncoder()
x[:,2] = labelencoder_x2.fit_transform(x[:,2])
labelencoder_x0 = LabelEncoder()
x[:,0] = labelencoder_x0.fit_transform(x[:,0])
x[1]

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
  
x = np.array(columnTransformer.fit_transform(x), dtype = np.float32) 

In [ ]:
x.shape

In [ ]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers 
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks

In [ ]:
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K

p_min = 0.001
p_max = 0.999
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

In [ ]:
import keras.losses
keras.losses.custom_loss = logloss

In [ ]:
num_classes = 402

model = models.Sequential()

model.add(tf.keras.Input(shape=(x.shape[1],)))
model.add(layers.BatchNormalization())

model.add(layers.Dense(12340))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))


model.add(layers.Dense(4500))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dropout(0.4))


model.add(layers.Dense(1024))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dropout(0.3))


model.add(layers.Dense(512))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dropout(0.3))




model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())


model.add(layers.Dense(num_classes, activation="sigmoid"))

adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), optimizer= adam, metrics=logloss)


model.summary()

In [ ]:
def log_loss_metric(y_true, y_pred):
    metrics = []
    for _target in y.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels = [0,1]))
    return np.mean(metrics)

In [ ]:
class AccuracyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

history = AccuracyHistory()
earlyStopping = callbacks.EarlyStopping(monitor='val_logloss', patience=4 ,restore_best_weights = True
                                        ,min_delta=1e-5, verbose=0, mode='min')
mcp_save = callbacks.ModelCheckpoint('mamon-drugtryrnonescoredpre.hdf5', save_best_only=True, monitor='val_logloss', mode='min')
reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_logloss',patience=2, verbose=2,factor=0.25,min_lr=0.0000001)

In [ ]:
batch_size =32

epochs = 5
y2 = y2.astype('float64') 

print(y2.dtype)

In [ ]:
x_train2, x_val2, y_train2, y_val2 = train_test_split(x, y2, test_size = 0.3, random_state = 0)

In [ ]:
import time
millis = int(round(time.time() * 1000))
print("started at " , millis)
 
model.fit(x_train2, y_train2, 
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val2,y_val2),callbacks=[earlyStopping, mcp_save, reduce_lr_loss,history]
         )

In [ ]:
model.load_weights('mamon-drugtryrnonescoredpre.hdf5')


In [ ]:
model.weights

In [ ]:
model.summary()


In [ ]:
num_classes = 206

modelscored = models.Sequential()
modelscored.add(tf.keras.Input(shape=(x.shape[1],)))
for layer in model.layers[:-1]: # go through until last layer
    modelscored.add(layer)

modelscored.add(layers.Dense(num_classes, activation="sigmoid"))
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
modelscored.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), optimizer= adam, metrics=logloss)

In [ ]:
modelscored.summary()


In [ ]:
modelscored.weights

In [ ]:
class AccuracyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

history = AccuracyHistory()
earlyStopping = callbacks.EarlyStopping(monitor='val_logloss', patience=6 ,restore_best_weights = True
                                        ,min_delta=1e-5, verbose=0, mode='min')
mcp_save = callbacks.ModelCheckpoint('mamon-drugtryr3.hdf5', save_best_only=True, monitor='val_logloss', mode='min')
reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_logloss',patience=3, verbose=2,factor=0.25,min_lr=0.0000001)

In [ ]:
batch_size =32

epochs = 100
y = y.astype('float64') 

print(y.dtype)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.3, random_state = 0)


In [ ]:
import time
millis = int(round(time.time() * 1000))
print("started at " , millis)
 
modelscored.fit(x_train, y_train, 
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val,y_val),callbacks=[earlyStopping, mcp_save, reduce_lr_loss,history]
         )



In [ ]:
acc = history.acc
val_acc = history.val_acc
loss = history.loss
val_loss = history.val_loss
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
# Loads the weights
modelscored.load_weights('mamon-drugtryr3.hdf5')

# Re-evaluate the model
loss, acc = modelscored.evaluate(x_val,y_val, verbose=2)
print("Restored model, accuracy: {:5.5f}%".format(  acc))

In [ ]:
testdrug_featuers = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
testdrug_featuers.head()

In [ ]:
fet_g = list(testdrug_featuers)[4:776]
fet_c = list(testdrug_featuers)[776:876]
print(testdrug_featuers.shape)
testdrug_featuers['gsum'] = testdrug_featuers[fet_g].sum(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['gstd'] = testdrug_featuers[fet_g].std(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['gmean'] = testdrug_featuers[fet_g].mean(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['gkurtosis'] = testdrug_featuers[fet_g].kurtosis(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['gskew'] = testdrug_featuers[fet_g].skew(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['csum'] = testdrug_featuers[fet_c].sum(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['cstd'] = testdrug_featuers[fet_c].std(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['cmean'] = testdrug_featuers[fet_c].mean(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['ckurtosis'] = testdrug_featuers[fet_c].kurtosis(axis=1)
print(testdrug_featuers.shape)
testdrug_featuers['cskew'] = testdrug_featuers[fet_c].skew(axis=1)
print(testdrug_featuers.shape)

In [ ]:
ind_te = testdrug_featuers[testdrug_featuers['cp_type']=='ctl_vehicle'].index
ind_te

In [ ]:
testdrug_featuers.shape

In [ ]:
print(testdrug_featuers.shape)

In [ ]:
sigid = testdrug_featuers.iloc[:, :1].values

sigid

In [ ]:
xtest = testdrug_featuers.iloc[:, 1:].values

xtest

In [ ]:
xtest.shape

In [ ]:
xtest[:,1] = xtest[:,1] / 72
labelencoder_xtest = LabelEncoder()
xtest[:,2] = labelencoder_xtest.fit_transform(xtest[:,2])
labelencoder_xtest0 = LabelEncoder()
xtest[:,0] = labelencoder_xtest0.fit_transform(xtest[:,0])
xtest[1]

In [ ]:
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
  
xtest = np.array(columnTransformer.fit_transform(xtest), dtype = np.float32) 
xtest

In [ ]:
y_pred = modelscored.predict(xtest)
y_pred

In [ ]:
# Convert to submission format
y_trai4n = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv').drop(columns=['sig_id'])
col = y_trai4n.columns
y_pred_df = pd.DataFrame(y_pred, columns = col)
sig_id = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')[['sig_id']]
y_pred_df = sig_id.join(y_pred_df)

y_pred_df.head(200)


In [ ]:
y_pred_df.loc[ind_te, col] = 0

In [ ]:
y_pred_df.head(200)


In [ ]:
#write to output
y_pred_df.to_csv('submission.csv', index=False)